In [1]:
import warnings
import tensorflow as tf
import os

warnings.filterwarnings("ignore", category=FutureWarning)

In [8]:
model_dir = '/scratch/sk7898/quantization/pipeline_cmsis/'
model_ts = '20200220101611'
output_dir = os.path.join(model_dir, model_ts)
tflite_path = os.path.join(output_dir, 'qa_cmsis_model.tflite')

# A list of the names of the model's input tensors
input_arrays = ['input_1']
# A list of the names of the model's output tensors
output_arrays = ['urban_sound_classifier/output/Sigmoid']

converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(os.path.join(output_dir, \
                                                                             'frozen_pipeline_cmsis_mels_quant.pb'),
                                                               input_arrays = input_arrays,\
                                                               output_arrays = output_arrays)

#converter.inference_type = tf.int8
#converter.inference_input_type = tf.int8
input_arrays = converter.get_input_arrays()
converter.default_ranges_min=0
converter.default_ranges_max=6

flatbuffer = converter.convert()

with open(tflite_path, 'wb') as outfile:
    outfile.write(flatbuffer)

In [9]:
interpreter = tf.lite.Interpreter(model_path=tflite_path) 
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape'][1:]
output_shape = output_details[0]['shape'][1:]
input_index = input_details[0]['index']
output_index = output_details[0]['index']

print("== Input details ==")
print(interpreter.get_input_details()[0])
print("type:", input_details[0]['dtype'])
print("\n== Output details ==")
print(interpreter.get_output_details()[0])

interpreter.allocate_tensors()

== Input details ==
{'name': 'input_1', 'index': 66, 'shape': array([ 1, 64, 51,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}
type: <class 'numpy.float32'>

== Output details ==
{'name': 'urban_sound_classifier/output/Sigmoid', 'index': 85, 'shape': array([], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}
